# MW Phillips Auction Scraping
The following is the code used to scrape auction data from the given sites (Section 1. Phillips Data Scraping), as well as the code (Section 2. Cleaning Artist Years Alive Table) used to clean table 1 (Data Strategy Art Market Case Study Data Tables 2 and 3/Data Table 2-Table 1.csv) which provides the lifespan of certain artists.
* "https://www.phillips.com/auctions/auction/UK010122"
* "https://www.phillips.com/auctions/auction/UK030122"
* "https://www.phillips.com/auctions/auction/HK010321"
* "https://www.phillips.com/auctions/auction/NY010721"
* "https://www.phillips.com/auctions/auction/UK010921"

The code extracts the following fields and performs preliminary data cleaning and organizes the data into a DataFrame that is exported to a .csv file for it to be imported into Airtable.
* "Artist Name",
*    "Title of Work",
*   "Date of Sale",      
*   "Local Currency",       
*   "Low Estimate (USD)",
*   "High Estimate (USD)",
*   "Transaction Price (USD)",
* "Source" (the website where the instance


In [1]:
import pandas as pd
import numpy as np
import datetime
from random import random
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import re
import dateutil.parser as dparser

## 1. Phillips Data Scraping

In [2]:
## Dictionary to convert currency symbols to more readable format
currency_dict = {
    "$": "USD"
    ,"£": "GBP"
    ,"€": "EUR"
    ,"฿": "BTC"
    ,"HK$": "HKD"
}

exchange_rate_dict = {
    "USD": 1.00
    ,"GBP": 1.22
    ,"EUR": 1.02
    ,"HKD": 0.13
}

## Fields of interest for our DataFrame

    # Looking for numbers/rankings for 
        # Longevity
        # Sillage
        # Price/Value
        # Gender
        # Overall rating
        # Day, Night, WSSF/Seasons
    # Also want
        # Note pyramid (list)
        # Main
    # Main Attributes
        # for women and men, men, women
        # designer

fields = [
    "Longevity",
    "Sillage",
    "",      
    "Local Currency",       
    "Low Estimate (USD)",
    "High Estimate (USD)",
    "Transaction Price (USD)",
    "Source",
]

## List of websites to scrape
df = pd.DataFrame(columns=[fields])
site_list = [
    "https://www.phillips.com/auctions/auction/UK010122"
    ,"https://www.phillips.com/auctions/auction/UK030122"
    ,"https://www.phillips.com/auctions/auction/HK010321"
    ,"https://www.phillips.com/auctions/auction/NY010721"
    ,"https://www.phillips.com/auctions/auction/UK010921"
]

AttributeError: type object 'object' has no attribute 'dtype'

In [8]:
fields

['Longevity',
 'Sillage',
 '',
 'Local Currency',
 'Low Estimate (USD)',
 'High Estimate (USD)',
 'Transaction Price (USD)',
 'Source']

In [11]:
page = requests.get("https://www.phillips.com/auctions/auction/UK010122")
soup = BeautifulSoup(page.content, 'html.parser')

In [13]:
soup.select('p.phillips-lot__description__title')

[<p class="phillips-lot__description__title" title="Airsickness"><em>Airsickness</em></p>,
 <p class="phillips-lot__description__title" title="Amy in Her Studio"><em>Amy in Her Studio</em></p>,
 <p class="phillips-lot__description__title" title="Ubuhle beenkanyezi VIII"><em>Ubuhle beenkanyezi VIII</em></p>,
 <p class="phillips-lot__description__title" title="The Apprentice Bench"><em>The Apprentice Bench</em></p>,
 <p class="phillips-lot__description__title" title="Balcones"><em>Balcones</em></p>,
 <p class="phillips-lot__description__title" title="My Blanket has a Possessive Nature"><em>My Blanket has a Possessive Nature</em></p>,
 <p class="phillips-lot__description__title" title="Chalet"><em>Chalet</em></p>,
 <p class="phillips-lot__description__title" title="Sylvia (87 Sylvia)"><em>Sylvia (87 Sylvia)</em></p>,
 <p class="phillips-lot__description__title" title="Houses"><em>Houses</em></p>,
 <p class="phillips-lot__description__title" title="Crooked"><em>Crooked</em></p>,
 <p class=

In [6]:

# For every website in the list of pages given to scrape

for site in site_list:
    # example_site = "https://www.phillips.com/auctions/auction/UK010122"
    page = requests.get(site)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for a in list(range(len(soup.select("p.phillips-lot__description__title")))):
    

        ## ARTWORK TITLE ##
        artwork_title = soup.select("p.phillips-lot__description__title")[a].text

        ## ARTIST NAME ##
        artist_name = soup.select("p.phillips-lot__description__artist")[a].text
        
        # Try t
        try:
                        
            ## LOCAL CURRENCY ##
            s = soup.select("p.phillips-lot__sold")[a].text.split("\xa0")[1]
            digit_ind = re.search(r"\d", s).start()
            
            # Convert currency symbol to acronym
            local_currency = currency_dict[s[0:digit_ind]]
            # Use the currency symbol
    #         local_currency = s[0:digit_ind]
            
            ## LOW and HIGH estimates
            estimate_range = soup.select("p.phillips-lot__description__estimate")[a].text[10:].split("\xa0-\xa0")
            
            # LOW #
            low_estimate = exchange_rate_dict[local_currency]*float(re.sub("[^0-9]", "", estimate_range[0]))

            # HIGH #
            high_estimate = exchange_rate_dict[local_currency]*float(re.sub("[^0-9]", "", estimate_range[1]))
            
            ## TRANSACTION PRICE ##
            sold_for_text = soup.select("p.phillips-lot__sold")[a].text[10:]
            price_start_i = re.search(r"\d",sold_for_text).start()
            sold_for_amount = exchange_rate_dict[local_currency]*float(re.sub("[^0-9]", "",sold_for_text[price_start_i:]))

            ## TRANSACTION DATE ##
            
            transaction_date = soup.select("p")[0].text[8:]
            
        except IndexError:
            
            ## LOW and HIGH estimates
            estimate_range = 'N/A'
            
            # LOW #
            low_estimate = 'N/A'

            # HIGH #
            high_estimate = 'N/A'
            
            sold_for_text = 'N/A'
            price_start_i = 'N/A'
            sold_for_amount = 'N/A'

            # Convert currency symbol to acronym
            local_currency = 'N/A'

            ## TRANSACTION DATE ##
            transaction_date = 'N/A' 
            
        # Looking for numbers/rankings for 
            # Longevity
            # Sillage
            # Price/Value
            # Gender
            # Overall rating
            # Day, Night, WSSF/Seasons
        # Also want
            # Note pyramid (list)
            # Main
        # Main Attributes
            # for women and men, men, women
            # designer
        
        data_list = [
            artist_name
            ,artwork_title
            ,transaction_date
            ,local_currency
            ,low_estimate
            ,high_estimate
            ,sold_for_amount
            ,site
        ]
        df = df.append(pd.DataFrame([data_list], 
         columns=fields), 
         ignore_index=True)


NameError: name 'df' is not defined

In [237]:
df.to_csv('/Users/justokarell/Desktop/PhillipsArtScraping.csv')


## 2. Cleaning Artist Years Alive Table
(Data Strategy Art Market Case Study Data Tables 2 and 3/Data Table 2-Table 1.csv)

In [238]:
## Cleaning the artist-years table
df_table1 = pd.read_csv('/Users/justokarell/Desktop/Data Strategy Art Market Case Study Data Tables 2 and 3/Data Table 2-Table 1.csv')

# Grab the first row for the header
new_header = df_table1.iloc[0]

# Take the data less the header row
df_table1 = df_table1[1:] 

# Set the header row as the df header
df_table1.columns = new_header 

df_table1['Years Alive'] = [df_table1['Artist Name'].to_list()[a].split('(')[1].rstrip(')') for a in range(len(df_table1))]
df_table1['Artist Name'] = [df_table1['Artist Name'].to_list()[a].split('(')[0].title().rstrip(' ').lstrip(' ') for a in range(len(df_table1))]
df_table1 = df_table1[['Artist Name', 'Years Alive']]

# Converting roman characters to numbersM
def romanToInt(s):
    sum=0
    dict={'M':1000,'D':500,'C':100,'L':50,'X':10,'V':5,'I':1}

    for i in range(len(s)):
        if i==0:
            sum=sum+dict[s[i]]
        else:
            if s[i]=='M':
                sum=sum+1000
                if s[i-1]=='C':
                    sum=sum-200      

            elif s[i]=='D':
                sum=sum+500
                if s[i-1]=='C':
                    sum=sum-200

            elif s[i]=='C':
                sum=sum+100
                if s[i-1]=='X':
                    sum=sum-20

            elif s[i]=='L':
                sum=sum+50
                if s[i-1]=='X':
                    sum=sum-20

            elif s[i]=='X':
                sum=sum+10
                if s[i-1]=='I':
                    sum=sum-2

            elif s[i]=='V':
                sum=sum+5
                if s[i-1]=='I':
                    sum=sum-2
            elif s[i]=='I':
                sum=sum+1
    return (sum)

# Converting a roman numeral to a year if it has a roman numeral in the specific format
# If there are two digits or less after the conversion, we assume it's a century
# This function assumes the data is striped (on the right and left) of auxilliary, lowercase characters like "b."

def RomToNum(b_year, century=False):        
# If there are any roman numerals in the year of birth
    rome_list =['M', 'D', 'C', 'L', 'X', 'V', 'I']
    if any(x in rome_list for x in b_year):

        # Convert it to an integer
        int_year = romanToInt(b_year)
        
        # If we're assuming that a lack of digits means that the number is a century, add digits to the end of it
        if century == True:

            # if the length of the year is 4
            if len(str(int_year))==4:
                # Do nothing, and make it the year of birth 
                b_year = int(str(int_year))

            # if the length of the year is 1
            if len(str(int_year))==1:
                # add 3 zeros to the end of it 
                b_year = int(str(int_year)+"000")

            # if the length of the year is 2
            if len(str(int_year))==2:
                # add 2 zeros to the end of it 
                b_year = int(str(int_year)+"00")

            # if the length of the year is 3
            elif len(str(int_year))==3:
                # add 1 zeros to the end of it 
                b_year = int(str(int_year)+"0")
                
        else:
            b_year = romanToInt(b_year)
    
    return b_year



# Check if a year has special characters that would make the year worth looking at a second time or deeming invalid

def CheckSpecialChar(year):
    
    special_chars = [
    '!'
    ,'?'
    ,'#'
    ,'%'
    ,'@'
    ,'*'
    ,'$'
    ,'^'
    ,'<'
    ,'>'
    ,'~'
    ,'`'
    ,'+'
    ,'_'
    ,'='
    ,'['
    ,']'
    ,'{'
    ,'}'
    ,'|'
    ,'.'
    ,':'
    ,';'
    ]
    
    if any(a in special_chars for a in str(year)):
        result = True
    else:
        result = False
    
    return result


l = [a.split('-') for a in df_table1['Years Alive']]
m = [RomToNum(b[0].lstrip('b.').lstrip('c.'),True) for b in l]
n = []
for c in m:
    if CheckSpecialChar(str(c))==True:
        f='Unknown'
        n.append(f)
    elif '/' in str(c):
        # choose the first one
        f = c.split('/')[0]
        n.append(int(f))
    else:
        n.append(c)
# print(len(n))

o = [a.split('-') for a in df_table1['Years Alive']]
q = []
for s in range(len(o)):
    # If there was only a birth date, we say they're alive
    if len(l[s])==1 and len(o[s])==1:
        q.append('Alive')
    # if there is no death date
    elif len(str(o[s][1]))==0:
        q.append('Unknown')
    else:
        if CheckSpecialChar(o[s][1].lstrip('b.').lstrip('c.'))==True:
            c='Unknown'
            q.append(c)
        else:
            c = RomToNum(o[s][1].lstrip('b.').lstrip('c.').split('/')[0])
            q.append(int(c))

# Cleaning the dates; turning all numerical values into integers and leaving strings alone
for a in range(len(n)):
    try:
        n[a] = int(n[a])
    except ValueError:
        pass
    
for a in range(len(q)):
    try:
        q[a] = int(q[a])
    except ValueError:
        pass

df_table1['Year Born'] = n
df_table1['Year Dead'] = q

## Decades and centuries that each artist was born

decade_list = []
century_list = []
for a in range(len(df_table1)):
    year = df_table1['Year Born'].to_list()[a]
    if type(year)==int:
        decade = str(year)[:3]+'0'
        century = str(year)[:2]+'00'
        decade_list.append(decade)
        century_list.append(century)
    else:
        decade_list.append(year)
        century_list.append(year)

df_table1['Decade Born'] = decade_list
df_table1['Century Born'] = century_list


## Decades and centuries that each artist was dead

decade_list = []
century_list = []
for a in range(len(df_table1)):
    year = df_table1['Year Dead'].to_list()[a]
    if type(year)==int:
        decade = str(year)[:3]+'0'
        century = str(year)[:2]+'00'
        decade_list.append(decade)
        century_list.append(century)
    else:
        decade_list.append(year)
        century_list.append(year)

df_table1['Decade Dead'] = decade_list
df_table1['Century Dead'] = century_list

# Exporting to csv titled 'artist_years.csv'
df_table1.to_csv('/Users/justokarell/Desktop/artist_years.csv')

In [239]:
## Creating a separate table that relates artwork to dates of birth and death for analysis

dfg = pd.merge(df, df_table1 , on='Artist Name', how='outer').drop_duplicates()
dfg = dfg[
    (dfg['Title of Work'].notna()) &
    (dfg['Date of Sale'].notna()) &
    (dfg['Years Alive'].notna())].drop_duplicates()
dfg.index = range(len(dfg))

# Exporting to csv titled 'price_life.csv'
dfg.to_csv('/Users/justokarell/Desktop/price_life.csv')

## Other things to consider

Other ways to clean the data scraped from Phillips include the following, some of which were attempted:
* Converting the dates to EST (attempted)
* averaging the years with a '/' (forward slash) between them (attempted)
* Fuzzy string matching of artist names with those in the third table
* Fuzzy string matching of artist names with those in an external database/source
* Account for a mix of characters in the years (e.g. '1C00' or 'MM200') although this is extremely difficult

In [240]:
### IGNORE ### 


# timezone_index = soup.select('script')[20].text.index('timeZone')
# s = soup.select('script')[20].text[timezone_index:timezone_index+35]
# m = s.index(',')
# l = s[:m].split(':')[1].strip(' "" ')


# # born_year_list = []
# # dead_year_list = []

# # # df_table1['Artist Name']
# # for a in range(1,len(df_table1)+1):

# #     # The years an artist were alive are split by the '-'
# #     life_range = df_table1['Years Alive'][a].split('-')

# #     # If there is no '-'
# #     if len(life_range) == 1:
        
# #         # If there is no '-' and the only entry in the "range" is empty
# #         if len(life_range[0])==0:
# #             # we say that both are unknown
# #             born_year = 'Unknown'
# #             dead_year = 'Unknown'
# #             born_year_list.append(born_year)
# #             dead_year_list.append(dead_year)

# #         # If there is at least one date
# #         elif len(life_range[0]) > 0:
            
# #             # we assume it's a birth year
# #             if 'b.' is not in life_range[0]
# #                 dead_year = d_year:
# #                 dead_year_list.append(dead_year)
            
# #             ### handling the first entry
# #             # we take anything to the left of the '-', or just a single year with no '-'
# #             b_year = life_range[0].lstrip('b.').lstrip('c.')

# #             # If there are weird/special characters in the year we say that it's  unknown
# #             if CheckSpecialChar(b_year) == True:
# #                 born_year = 'Unknown'
# #                 born_year_list.append(born_year)

# #             ## If there is a "/" in the birth year handle it differently
# #             elif "/" in b_year:

# #                 # If there are multiple '/'s then we say that it's unknown
# #                 if len(b_year.split("/")) > 2:
# #                     born_year = 'Unknown'
# #                     born_year_list.append(born_year)

# #                 # Check if there's only 1 "/", then we take the first one
# #                 elif len(b_year.split("/")) == 2:
# #                     born_year = RomToNum(b_year[0].lstrip('b.').lstrip('c.'))
# #                     born_year_list.append(born_year)
                    
# #             # if there's nothing weird about the string
# #             else:
# #                 born_year = b_year
# #                 born_year_list.append(born_year)

                    
# #     elif len(life_range) == 2:

# #         # If there is at least one date
# #         if len(life_range[0]) > 0:

# #             ### handling the first entry
# #             # we take anything to the left of the '-', or just a single year with no '-'
# #             b_year = life_range[0].lstrip('b.').lstrip('c.')

# #             # If there are weird/special characters in the year we say that it's  unknown
# #             if CheckSpecialChar(b_year) == True:
# #                 born_year = 'Unknown'
# #                 born_year_list.append(born_year)

# #             ## If there is a "/" in the birth year handle it differently
# #             if "/" in b_year:

# #                 # If there are multiple '/'s then we say that it's unknown
# #                 if len(b_year.split("/")) > 2:
# #                     born_year = 'Unknown'
# #                     born_year_list.append(born_year)

# #                 # Check if there's only 1 "/", then we take the first one
# #                 elif len(b_year.split("/")) == 2:
# #                     born_year = RomToNum(b_year[0].lstrip('b.').lstrip('c.'))
# #                     born_year_list.append(born_year)


# #             # if there's nothing weird about the string
# #             else:
# #                 born_year = RomToNum(b_year.lstrip('b.').lstrip('c.'))
# #                 born_year_list.append(born_year)
            
# #                 # check if there are any roman numerals in it, and convert to integer
# #                 rome_list =['M', 'D', 'C', 'L', 'X', 'V', 'I']
# #                 if any(x in rome_list for x in b_year.lstrip('b.').lstrip('c.')):
                    
# #                     # check if roman numerals in first part of '/' split
# #                     if any(x in rome_list for x in b_year.split("/")[0].lstrip('b.').lstrip('c.')):                        
# #                         # convert the numbers from Roman to regular
# #                         # we assume that this number at least includes a century
# #                         b_year1 = RomToNum(b_year.split("/")[0].lstrip('b.').lstrip('c.'), True)

# #                     # check if roman numerals in second part of '/' split
# #                     if any(x in rome_list for x in b_year.split("/")[1].lstrip('b.').lstrip('c.')):
# #                         # we assume the second part does not include a century
# #                         # If the number has >1 digits
# #                         if len(str(RomToNum(b_year.split("/")[1].lstrip('b.').lstrip('c.')))) > 1:
# #                             b_year2 = RomToNum(b_year.split("/")[1].lstrip('b.').lstrip('c.'))
# #                         else:
# #                             b_year2 = int(str(b_year1[:-2])+str(RomToNum(b_year.split("/")[1].lstrip('b.').lstrip('c.'))))
                

# #                 # otherwise average the two numbers across the '/' 
# #                 else:
# #                     # number of digits to take away from first part of '/' to indicate the century
# #                     j = len(b_year[0])
# #                     k = len(b_year[1]
# #                     # If the two numbers have the same number of digits we just average 
# #                     if k==j:
# #                         born_year = (b_year[0]+b_year[1])/2
# #                     # if the second part has less digits than the first
# #                     elif k<j:
# #                         o = j-k
# #                         # the start is the first number of digits till the difference in digits
# #                         # e.g. b.1531/32 --> start = 15,  e.g. b.531/2 --> start = 53
# #                         start = b_year[0][:o]
# #                             year1 = str(start)+str(b_year[0][])
# #                         year1 = int(b_year.split("/")[0][2:4])
# #                         year2 = int(b_year.split("/")[0][-2:])
# #                         average = int((year1 + year2)/2)
# #                         dead_year = century+str(average)

# #                 born_year_list.append(born_year)     


            
#         # If there's a '-' and two entries
#         if len(life_range) == 2:
#             ### handling the first entry
#             # we take anything to the left of the '-', or just a single year with no '-'
#             b_year = life_range[0].lstrip('b.').lstrip('c.')

#             # If there are weird/special characters in the year we say that it's  unknown
#             if CheckSpecialChar(b_year) == True:
#                 born_year = 'Unknown'
#                 born_year_list.append(born_year)

#             ## If there is a "/" in the birth year handle it differently
#             if "/" in b_year:

#                 # If there are multiple '/'s then we say that it's unknown
#                 if len(b_year.split("/")) > 2:
#                     born_year = 'Unknown'
#                     born_year_list.append(born_year)

#                 # Check if there's only 1 "/", then we take the first one
#                 elif len(b_year.split("/")) == 2:
#                     born_year = RomToNum(b_year[0].lstrip('b.').lstrip('c.'))
#                     born_year_list.append(born_year)


#             # if there's nothing weird about the string
#             else:
#                 born_year = RomToNum(b_year.lstrip('b.').lstrip('c.'))
#                 born_year_list.append(born_year)
            
#             ### handling the second entry
#             # we take anything to the left of the '-', or just a single year with no '-'
#             d_year = life_range[1].lstrip('b.').lstrip('c.')

#             # If there are weird/special characters in the year we say that it's  unknown
#             if CheckSpecialChar(b_year) == True:
#                 dead_year = 'Unknown'
#                 dead_year_list.append(dead_year)

#             ## If there is a "/" in the birth year handle it differently
#             if "/" in d_year:

#                 # If there are multiple '/'s then we say that it's unknown
#                 if len(d_year.split("/")) > 2:
#                     dead_year = 'Unknown'
#                     dead_year_list.append(dead_year)

#                 # Check if there's only 1 "/", then we take the first one
#                 elif len(b_year.split("/")) == 2:
#                     dead_year = RomToNum(d_year[0].lstrip('b.').lstrip('c.'))
#                     dead_year_list.append(dead_year)


#             # if there's nothing weird about the string
#             else:
#                 dead_year = RomToNum(d_year.lstrip('b.').lstrip('c.'))
#                 dead_year_list.append(dead_year)
            
            